<a href="https://colab.research.google.com/github/harnalashok/LLMs/blob/main/RAG_on_Colab_with_Huggingface_and_langchain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Last amended: 29th June, 2024
# Ref: https://huggingface.co/learn/cookbook/en/rag_zephyr_langchain

# Simple RAG for GitHub issues using Hugging Face Zephyr and LangChain

_Authored by: [Maria Khalusova](https://github.com/MKhalusova)_

This notebook demonstrates how one can quickly build a RAG (Retrieval Augmented Generation) for a project's GitHub issues using [`HuggingFaceH4/zephyr-7b-beta`](https://huggingface.co/HuggingFaceH4/zephyr-7b-beta) model, and LangChain.

## What is RAG

RAG is a popular approach to address the issue of a powerful LLM not being aware of specific content due to said content not being in its training data, or hallucinating even when it has seen it before. Such specific content may be proprietary, sensitive, or, as in this example, recent and updated often.

If your data is static and doesn't change regularly, you may consider fine-tuning a large model. In many cases, however, fine-tuning can be costly, and, when done repeatedly (e.g. to address data drift), leads to "model shift". This is when the model's behavior changes in ways that are not desirable.

**RAG (Retrieval Augmented Generation)** does not require model fine-tuning. Instead, RAG works by providing an LLM with additional context that is retrieved from relevant data so that it can generate a better-informed response.

Here's a quick illustration:

![RAG diagram](https://huggingface.co/datasets/huggingface/cookbook-images/resolve/main/rag-diagram.png)

* The external data is converted into embedding vectors with a separate embeddings model, and the vectors are kept in a database. Embeddings models are typically small, so updating the embedding vectors on a regular basis is faster, cheaper, and easier than fine-tuning a model.

* At the same time, the fact that fine-tuning is not required gives you the freedom to swap your LLM for a more powerful one when it becomes available, or switch to a smaller distilled version, should you need faster inference.

Let's illustrate building a RAG using an open-source LLM, embeddings model, and LangChain.

First, install the required dependencies:

## Install software

In [ ]:
# 0.0
!pip install -q accelerate bitsandbytes transformers sentence-transformers faiss-gpu

In [ ]:
# 0.1 If running in Google Colab, you may need to run this
#       cell to make sure you're using UTF-8 locale
#       to install LangChain

import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
# 0.2 Install langchain

!pip install -q langchain langchain-community

## Download and Load GitHub data


In this example, we'll load all of the issues (both open and closed) from [PEFT library's repo](https://github.com/huggingface/peft).

First, you need to acquire a [GitHub personal access token](https://github.com/settings/tokens?type=beta) to access the GitHub API.

In [ ]:
# 0.3 Access to github repo:

from getpass import getpass
ACCESS_TOKEN = getpass("YOUR_GITHUB_PERSONAL_TOKEN")

YOUR_GITHUB_PERSONAL_TOKEN··········


Next, we'll load all of the issues in the [huggingface/peft](https://github.com/huggingface/peft) repo:
- By default, pull requests are considered issues as well, here we chose to exclude them from data with by setting `include_prs=False`
- Setting `state = "all"` means we will load both open and closed issues.

In [ ]:
%%time

# 1.0 Load github issues:

from langchain.document_loaders import GitHubIssuesLoader

# 1.0.1 Instantiate github issues loader:

loader = GitHubIssuesLoader(
                            repo="huggingface/peft",
                            access_token=ACCESS_TOKEN,
                            include_prs=False,
                            state="all"
                           )


In [ ]:
%%time

# 1.0.2 Load the documents:

docs = loader.load()

## Splitting and Chunking
Data once loaded is in text form. Split and chunk it

The content of individual GitHub issues may be longer than what an embedding model can take as input. If we want to embed all of the available content, we need to chunk the documents into appropriately sized pieces.

The most common and straightforward approach to chunking is to define a fixed size of chunks and whether there should be any overlap between them. Keeping some overlap between chunks allows us to preserve some semantic context between the chunks. The recommended splitter for generic text is the [RecursiveCharacterTextSplitter](https://python.langchain.com/docs/modules/data_connection/document_transformers/recursive_text_splitter), and that's what we'll use here.

In [ ]:
%%time

# 2.0 Split documents into chunks:
from langchain.text_splitter import RecursiveCharacterTextSplitter

# 2.0.1 Instantiate the splitter class:
splitter = RecursiveCharacterTextSplitter(chunk_size=512,
                                          chunk_overlap=30
                                          )



CPU times: user 534 ms, sys: 6.77 ms, total: 541 ms
Wall time: 585 ms


In [ ]:
%%time

# 2.0.2 Chunk docs now:

chunked_docs = splitter.split_documents(docs)

## Create the embeddings

Now that the docs are all of the appropriate size, we can create a database with their embeddings.

To create document chunk embeddings we'll use the `HuggingFaceEmbeddings` and the [`BAAI/bge-base-en-v1.5`](https://huggingface.co/BAAI/bge-base-en-v1.5) embeddings model. There are many other embeddings models available on the Hub, and you can keep an eye on the best performing ones by checking the [Massive Text Embedding Benchmark (MTEB) Leaderboard](https://huggingface.co/spaces/mteb/leaderboard).


To create the vector database, we'll use `FAISS`, a library developed by Facebook AI. This library offers efficient similarity search and clustering of dense vectors, which is what we need here. FAISS is currently one of the most used libraries for NN search in massive datasets.

We'll access both the embeddings model and FAISS via LangChain API.

In [ ]:
%%time

# 3.0 Call libraries:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings

# 3.0.1 Create database of vectors:
db = FAISS.from_documents(chunked_docs,
                          HuggingFaceEmbeddings(
                                                 model_name='BAAI/bge-base-en-v1.5'
                                                )
                          )

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_downlo

CPU times: user 1min 6s, sys: 1.63 s, total: 1min 7s
Wall time: 1min 12s


The vector database is now set up, next we need to set up the next piece of the chain - the model.

## Load quantized model

For this example, we chose [`HuggingFaceH4/zephyr-7b-beta`](https://huggingface.co/HuggingFaceH4/zephyr-7b-beta), a small but powerful model. It's model card is [here](https://huggingface.co/HuggingFaceH4/zephyr-7b-beta)

With many models being released every week, you may want to substitute this model to the latest and greatest. The best way to keep track of open source LLMs is to check the [Open-source LLM leaderboard](https://huggingface.co/spaces/HuggingFaceH4/open_llm_leaderboard).

To make inference faster, we will load the quantized version of the model:

In [ ]:
%%time

# 4.0.1 Call libraries
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

# 4.0.2 Which model to reduce to 4bit quantized:
model_name = 'HuggingFaceH4/zephyr-7b-beta'

# 4.0.3 Configure for reduction:
bnb_config = BitsAndBytesConfig(
                                load_in_4bit=True,
                                bnb_4bit_use_double_quant=True,
                                bnb_4bit_quant_type="nf4",
                                bnb_4bit_compute_dtype=torch.bfloat16
                                )



`low_cpu_mem_usage` was None, now set to True since model is quantized.


model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

CPU times: user 35.3 s, sys: 37.4 s, total: 1min 12s
Wall time: 2min 38s


In [ ]:
%%time

# 5.0 Download model
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             quantization_config=bnb_config
                                             )

# 5.0.1
tokenizer = AutoTokenizer.from_pretrained(model_name)

## Setup the LLM chain

Finally, we have all the pieces we need to set up the LLM chain.

First, create a text_generation pipeline using the loaded model and its tokenizer.

Next, create a prompt template - this should follow the format of the model, so if you substitute the model checkpoint, make sure to use the appropriate formatting.

In [ ]:
%%time

from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from transformers import pipeline
from langchain_core.output_parsers import StrOutputParser

In [ ]:
text_generation_pipeline = pipeline(
                                    model=model,
                                    tokenizer=tokenizer,
                                    task="text-generation",
                                    temperature=0.2,
                                    do_sample=True,
                                    repetition_penalty=1.1,
                                    return_full_text=True,
                                    max_new_tokens=400,
                                  )

In [ ]:
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)

## Design prompt

Prompt for zephyr has this form:<br>
<|system|><br>
You are a friendly chatbot who always responds in the style of a pirate.</s><br>
<|user|><br>
How many helicopters can a human eat in one sitting?</s><br>
<|assistant|><br>
Ah, me hearty matey! But yer question be a puzzler! A human cannot eat a helicopter in one sitting, as helicopters are not edible. They be made of metal, plastic, and other materials, not food!<br>

In [ ]:
prompt_template = """
<|system|>
Answer the question based on your knowledge. Use the following context to help:

{context}

</s>
<|user|>
{question}
</s>
<|assistant|>

 """

In [ ]:
prompt = PromptTemplate(
                        input_variables=["context", "question"],
                        template=prompt_template,
                        )

In [ ]:
llm_chain = prompt | llm | StrOutputParser()

Note: _You can also use `tokenizer.apply_chat_template` to convert a list of messages (as dicts: `{'role': 'user', 'content': '(...)'}`) into a string with the appropriate chat format._


We need a way to return(retrieve) the documents given an unstructured query. For that, we'll use the `as_retriever` method using the `db` as a backbone:
- `search_type="similarity"` means we want to perform similarity search between the query and documents
- `search_kwargs={'k': 4}` instructs the retriever to return top 4 results.

In [ ]:
%%time

# 4.0 Instantiate retriever:
retriever = db.as_retriever(
                            search_type="similarity",
                            search_kwargs={'k': 4}
                           )

Finally, we need to combine the `llm_chain` with the retriever to create a RAG chain. We pass the original question through to the final generation step, as well as the retrieved context docs:

In [ ]:
%%time

from langchain_core.runnables import RunnablePassthrough

retriever = db.as_retriever()

rag_chain = (
             {"context": retriever, "question": RunnablePassthrough()}
             | llm_chain
             )


CPU times: user 537 µs, sys: 0 ns, total: 537 µs
Wall time: 543 µs


## Compare the results

Let's see the difference RAG makes in generating answers to the library-specific questions.

In [ ]:
question = "How do you combine multiple adapters?"

First, let's see what kind of answer we can get with just the model itself, no context added:

In [ ]:
%%time

llm_chain.invoke({"context":"", "question": question})

CPU times: user 30.8 s, sys: 344 ms, total: 31.1 s
Wall time: 32.1 s


"\n<|system|>\nAnswer the question based on your knowledge. Use the following context to help:\n\n\n\n</s>\n<|user|>\nHow do you combine multiple adapters?\n</s>\n<|assistant|>\n\n  To combine multiple adapters, you need to ensure that they are compatible with each other and with the devices you want to connect. Here's how you can do it:\n\n1. Identify the adapters you need: Determine which adapters will allow you to connect the devices you want to use together. For example, if you want to connect a USB-C device to an HDMI monitor, you might need a USB-C to HDMI adapter, as well as an HDMI to DVI adapter (if your monitor only supports DVI).\n\n2. Connect the first adapter: Plug the first adapter into the device you want to connect. For instance, if you're connecting a laptop to a projector, plug the USB-C to HDMI adapter into your laptop's USB-C port.\n\n3. Connect the second adapter: Next, connect the second adapter to the first one. In our example, you would connect the HDMI to DVI a

As you can see, the model interpreted the question as one about physical computer adapters, while in the context of PEFT, "adapters" refer to LoRA adapters.
Let's see if adding context from GitHub issues helps the model give a more relevant answer:

In [ ]:
%%time

rag_chain.invoke(question)

CPU times: user 36.2 s, sys: 2.34 s, total: 38.5 s
Wall time: 38.7 s


'\n<|system|>\nAnswer the question based on your knowledge. Use the following context to help:\n\n[Document(page_content=\'The documentation does not mention the need to perform a merge when switching adapters. Additionally, the methods add_adapter, set_adapter, and enable_adapters do not appear to work\\r\\n\\r\\nPlease provide clarification on how to correctly switch between adapters\', metadata={\'url\': \'https://github.com/huggingface/peft/issues/1802\', \'title\': \'Issues when switching between multiple adapters LoRAs \', \'creator\': \'JhonDan1999\', \'created_at\': \'2024-05-26T19:18:13Z\', \'comments\': 7, \'state\': \'open\', \'labels\': [], \'assignee\': None, \'milestone\': None, \'locked\': False, \'number\': 1802, \'is_pull_request\': False}), Document(page_content="If you can provide any advice, I would greatly appreciate it. I suspect that this is either unsupported and/or not fully-implemented; or, it has something to do with the way I\'m attaching adapters. I\'ve tri

As we can see, the added context, really helps the exact same model, provide a much more relevant and informed answer to the library-specific question.

Notably, combining multiple adapters for inference has been added to the library, and one can find this information in the documentation, so for the next iteration of this RAG it may be worth including documentation embeddings.